#### Soybean (Large) Model

Descrição: O dataset contém diversas informações sobre grãos de soja e deseja classifica-los em uma das 19 classes apresentadas (diaporthe-stem-canker, charcoal-rot, rhizoctonia-root-rot,
     phytophthora-rot, brown-stem-rot, powdery-mildew,
     downy-mildew, brown-spot, bacterial-blight,
     bacterial-pustule, purple-seed-stain, anthracnose,
     phyllosticta-leaf-spot, alternarialeaf-spot,
     frog-eye-leaf-spot, diaporthe-pod-&-stem-blight,
     cyst-nematode, 2-4-d-injury, herbicide-injury). São fornecidas 307 instâncias no total

Resultados: O modelo não se saiu muito bem nas métricas avaliadas. Para melhorar os resultados foi implementada uma substituição da "?" por dados sintéticos baseados na moda da coluna em questão que estava sendo avaliada. Os resultados negativos do modelo provavelmente se devem a baixa quantidade de amostras.

In [13]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


In [ ]:
soybean_large = fetch_ucirepo(id=90)
X_original = soybean_large.data.features
y_original = soybean_large.data.targets.iloc[:, 0]

# Substituir '?' por NaN
X_original = X_original.replace('?', np.nan)

# Preencher valores ausentes com o valor mais frequente de cada coluna (moda)
for col in X_original.columns:
    X_original[col] = X_original[col].fillna(X_original[col].mode()[0])

print(f"Dimensão inicial de X: {X_original.shape}")
# print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
# print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")
print("-" * 50)

Dimensão inicial de X: (307, 35)

Classes de y:
class
alternarialeaf-spot            40
brown-spot                     40
phytophthora-rot               40
frog-eye-leaf-spot             40
brown-stem-rot                 20
anthracnose                    20
diaporthe-stem-canker          10
rhizoctonia-root-rot           10
charcoal-rot                   10
downy-mildew                   10
powdery-mildew                 10
purple-seed-stain              10
bacterial-pustule              10
bacterial-blight               10
phyllosticta-leaf-spot         10
diaporthe-pod-&-stem-blight     6
cyst-nematode                   6
herbicide-injury                4
2-4-d-injury                    1
Name: count, dtype: int64
--------------------------------------------------


--- 1. Carregando e Inspecionando o Dataset Soybean ---
Dimensão inicial de X: (307, 35)

Classes de y:
class
alternarialeaf-spot            40
brown-spot                     40
phytophthora-rot               40
frog-eye-leaf-spot             40
brown-stem-rot                 20
anthracnose                    20
diaporthe-stem-canker          10
rhizoctonia-root-rot           10
charcoal-rot                   10
downy-mildew                   10
powdery-mildew                 10
purple-seed-stain              10
bacterial-pustule              10
bacterial-blight               10
phyllosticta-leaf-spot         10
diaporthe-pod-&-stem-blight     6
cyst-nematode                   6
herbicide-injury                4
2-4-d-injury                    1
Name: count, dtype: int64
--------------------------------------------------


In [ ]:
print("---  Pré-processamento: One-Hot Encoding ---")

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False retorna array denso

X_encoded_dense = encoder.fit_transform(X_original)

print(f"Dimensão de X após One-Hot Encoding: {X_encoded_dense.shape}")
print(f"Tipo dos elementos após One-Hot Encoding: {X_encoded_dense.dtype}")
print(f"Primeiras 2 linhas de X_encoded_dense:\n{X_encoded_dense[:2]}")
print("-" * 50)

--- 2. Pré-processamento: One-Hot Encoding ---
Dimensão de X após One-Hot Encoding: (307, 132)
Tipo dos elementos após One-Hot Encoding: float64
Primeiras 2 linhas de X_encoded_dense:
[[0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
  1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.
  1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.
  0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
  1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.
  1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.
  0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0.

In [24]:
# --- Divisão dos Dados em Treino e Teste ---
print("--- 3. Dividindo os Dados em Treino e Teste ---")

# Para o WiSARD, os dados de entrada X devem ser do tipo np.uint8.
# E os rótulos y devem ser strings.
# Fazemos a conversão de tipo após o split para manter a consistência do tipo de dados.
X_encoded_uint8 = X_encoded_dense.astype(np.uint8)
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")
print("-" * 50)

--- 3. Dividindo os Dados em Treino e Teste ---
Tamanho do conjunto de treino (X_train): 92 amostras, 132 bits
Tamanho do conjunto de teste (X_test): 215 amostras, 132 bits
Tipo dos elementos de X_train: uint8
--------------------------------------------------


In [26]:
print("--- Treinando o Modelo WiSARD ---")

# addressSize (tuple_size) para o WiSARD.
# O número de bits de entrada de X_encoded_dense para Car Evaluation é 21.
# Um addressSize de 21 ainda é muito grande.
# Escolha um addressSize menor, como 2, 3 ou 4.
# Por exemplo, se X_encoded_dense tem 21 bits, e addressSize=3, teremos 21/3 = 7 RAMs por classe.
addressSize = 3 

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)
print("-" * 50)


print("--- Classificação e Avaliação do Modelo WiSARD ---")

print("Classificando os dados de teste...")
predictions_str = wsd.classify(X_test)
print("Classificação concluída.")

# Convertendo as previsões de string de volta para um array NumPy para fácil comparação
predictions_np = np.array(predictions_str)
y_test_np = np.array(y_test) 

print("Relatório de Classificação:")
print(classification_report(
    y_test_np, predictions_np,
    labels=np.unique(y_test),
))

--- Treinando o Modelo WiSARD ---
WiSARD inicializado com addressSize=3
--------------------------------------------------
--- Classificação e Avaliação do Modelo WiSARD ---
Classificando os dados de teste...
training 57 of 92Classificação concluída.
Relatório de Classificação:
training 92 of 92
classifying 215 of 215
                             precision    recall  f1-score   support

               2-4-d-injury       0.00      0.00      0.00         1
        alternarialeaf-spot       0.48      0.82      0.61        28
                anthracnose       0.74      1.00      0.85        14
           bacterial-blight       1.00      0.57      0.73         7
          bacterial-pustule       1.00      0.83      0.91         6
                 brown-spot       0.88      0.74      0.81        31
             brown-stem-rot       0.78      1.00      0.88        14
               charcoal-rot       1.00      0.50      0.67         8
              cyst-nematode       1.00      1.00      1.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
